# 데이터 불러오기

In [3]:
import pandas as pd

DATA_TRAIN_PATH = "../../data/nlp/summ_train.json"
DATA_TEST_PATH = "../../data/nlp/summ_test.json"

train_df = pd.read_json(DATA_TRAIN_PATH)
train_df = train_df.dropna()
train_df = train_df[:40000]
import pandas as pd

DATA_TRAIN_PATH = "../../data/nlp/summ_train.json"
DATA_TEST_PATH = "../../data/nlp/summ_test.json"

train_df = pd.read_json(DATA_TRAIN_PATH)
train_df = train_df.dropna()
train_df = train_df[:40000]


test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
test_df = test_df[:5000]


test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
test_df = test_df[:5000]

In [5]:
def preprocess_data(data):
    outs = []
    for doc in data["documents"]:
        line = []
        line.append(doc["media_name"])
        line.append(doc["id"])
        para = []
        for sent in doc["text"]:
            for s in sent:
                para.append(s["sentence"])
        line.append(para)
        line.append(doc["abstractive"][0])
        line.append(doc["extractive"])
        a = doc["extractive"]
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)

    outs_df = pd.DataFrame(outs)
    outs_df.columns = ["media", "id", "article_original", "abstractive", "extractive"]
    return outs_df

In [6]:
# 원문과 요약문을 각각 'article_original'와 'abstractive'열에 저장.
train_data = preprocess_data(train_df)
train_data.head()

test_data = preprocess_data(test_df)
test_data.head()

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"
1,한국경제,340626896,"[] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다., 바른미래당이 8일 ...",8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,"[2, 1, 6]"
2,한국경제,340626904,"[[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀., 지난 3일 한국이 세계 최초...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,"[1, 5, 8]"
3,한국경제,340627450,[] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,"[0, 1, 2]"
4,한국경제,340627465,"[[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,"[1, 3, 4]"


In [7]:
train_data["news"] = train_data["article_original"].apply(lambda x: " ".join(x))
test_data["news"] = test_data["article_original"].apply(lambda x: " ".join(x))

# T5 데이터 세트 정의

In [8]:
import numpy as np
from torch.utils.data import DataLoader, Dataset


class T5SummaryDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, ignore_index=-100):
        super().__init__()
        self.tokenizer = tokenizer  # 입력 문장을 토큰화하기 위한 T5 토크나이저
        self.max_len = max_len  # 입력 및 출력 문장의 최대 길이
        self.docs = df  # 입력 데이터 프레임
        self.len = self.docs.shape[0]  # 데이터셋의 크기
        self.pad_index = (
            self.tokenizer.pad_token_id
        )  # 패딩 토큰의 인덱스 (기본적으로 0)
        self.ignore_index = (
            ignore_index  # 손실 계산 시 무시할 토큰의 인덱스 (기본적으로 -100)
        )

    def add_padding_data(self, inputs):
        """
        입력 시퀀스에 패딩을 추가하거나 자르기.
        입력 시퀀스가 max_len보다 짧으면 패딩을 추가하고, 길면 자릅니다.
        """
        if len(inputs) < self.max_len:
            # 시퀀스 길이가 max_len보다 짧으면 패딩 추가
            pad = np.array([self.pad_index] * (self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            # 시퀀스 길이가 max_len보다 길면 자르기
            inputs = inputs[: self.max_len]
        return inputs

    def add_ignored_data(self, inputs):
        """
        출력 시퀀스에 무시할 토큰(기본값: -100)을 추가하거나 자르기.
        출력 시퀀스가 max_len보다 짧으면 무시할 토큰을 추가하고, 길면 자릅니다.
        """
        if len(inputs) < self.max_len:
            # 시퀀스 길이가 max_len보다 짧으면 무시할 토큰 추가
            pad = np.array([self.ignore_index] * (self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            # 시퀀스 길이가 max_len보다 길면 자르기
            inputs = inputs[: self.max_len]
        return inputs

    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 데이터를 반환.
        뉴스 기사를 요약할 입력 텍스트와 대응하는 요약문을 반환합니다.
        """
        instance = self.docs.iloc[
            idx
        ]  # 데이터 프레임에서 해당 인덱스의 데이터를 가져오기
        input_text = (
            "summarize: " + instance["news"]
        )  # 입력 텍스트에 "summarize: " 접두사를 추가
        input_ids = self.tokenizer.encode(
            input_text, return_tensors="pt", max_length=self.max_len, truncation=True
        ).squeeze()
        input_ids = self.add_padding_data(input_ids)  # 입력 시퀀스에 패딩 추가

        label_ids = self.tokenizer.encode(
            instance["abstractive"],
            return_tensors="pt",
            max_length=self.max_len,
            truncation=True,
        ).squeeze()
        label_ids = self.add_ignored_data(label_ids)  # 출력 시퀀스에 무시할 토큰 추가

        return {
            "input_ids": np.array(input_ids, dtype=np.int_),  # 입력 시퀀스
            "labels": np.array(label_ids, dtype=np.int_),  # 출력 시퀀스 (요약문)
        }

    def __len__(self):
        """데이터셋의 총 데이터 수를 반환."""
        return self.len

# 모델 정의

In [ ]:
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration


class T5ConditionalGeneration(torch.nn.Module):
    def __init__(self):
        super(T5ConditionalGeneration, self).__init__()

        # T5 모델과 토크나이저 초기화 (사전 학습된 모델 사용)
        self.model = T5ForConditionalGeneration.from_pretrained("paust/pko-t5-base")
        self.tokenizer = T5TokenizerFast.from_pretrained("paust/pko-t5-base")

        # 패딩 토큰 ID 저장
        self.pad_token_id = self.tokenizer.pad_token_id

    def forward(self, inputs):
        # 패딩이 아닌 토큰에 대해 어텐션 마스크 생성
        attention_mask = inputs["input_ids"].ne(self.pad_token_id).float()

        # 모델의 forward 메서드 호출
        # 입력 시퀀스(input_ids), 어텐션 마스크(attention_mask), 레이블(labels)를 사용하여 예측 수행
        return self.model(
            input_ids=inputs["input_ids"],
            attention_mask=attention_mask,
            labels=inputs["labels"],
            return_dict=True,
        )


# GPU 사용 가능 여부에 따라 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 인스턴스 생성 및 디바이스로 이동
model = T5ConditionalGeneration().to(device)